# Add dates to the file list

Combine the list of files on CloudStor with the item list from the Archives catalogue so that we can easily tell which files belong to which year.

In [1]:
import pandas as pd

In [2]:
# Harvested from CloudStor
files_df = pd.read_csv('files.csv')

In [3]:
files_df.head()

,directory,name,path
0,AU NBAC N193-001/,N193-001_0001.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...
1,AU NBAC N193-001/,N193-001_0002.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...
2,AU NBAC N193-001/,N193-001_0003.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...
3,AU NBAC N193-001/,N193-001_0004.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...
4,AU NBAC N193-001/,N193-001_0005.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...


In [10]:
# From the catalogue
items_df = pd.read_csv('itemList.csv')

In [11]:
items_df

,referenceCode,title,dates,startDate,accessConditions,locations
0,N193-1,Sydney Stock Exchange Stock and Share Lists,1901-01-01 - 1901-03-01,1901-01-01,NaN,NaN
1,N193-2,Sydney Stock Exchange Stock and Share Lists,1901-04-01 - 1901-06-01,1901-04-01,NaN,NaN
2,N193-3,Sydney Stock Exchange Stock and Share Lists,1901-07-01 - 1901-09-01,1901-07-01,NaN,NaN
3,N193-4,Sydney Stock Exchange Stock and Share Lists,1901-10-01 - 1901-12-01,1901-10-01,NaN,NaN
4,N193-5,Sydney Stock Exchange Stock and Share Lists,1902-01-01 - 1902-03-01,1902-01-01,NaN,NaN
5,N193-6,Sydney Stock Exchange Stock and Share Lists,1902-04-01 - 1902-06-01,1902-04-01,NaN,NaN
6,N193-7,Sydney Stock Exchange Stock and Share Lists,1902-07-01 - 1902-09-01,1902-07-01,NaN,NaN
7,N193-8,Sydney Stock Exchange Stock and Share Lists,1902-10-01 - 1902-12-01,1902-10-01,NaN,NaN
8,N193-9,Sydney Stock Exchange Stock and Share Lists,1903-01-01 - 1903-03-01,1903-01-01,NaN,NaN
9,N193-10,Sydney Stock Exchange Stock and Share Lists,1903-04-01 - 1903-06-01,1903-04-01,NaN,NaN


In [12]:
# Reference codes in the Archives item list are not zero padded.
# Need to zero pad them so we can use them to link with the file list.
def pad_ids(ref_id):
    collection, item = ref_id.split('-')
    padded = item.zfill(3)
    return '{}-{}'.format(collection, padded)

items_df['referenceCode'] = items_df['referenceCode'].apply(pad_ids)

In [13]:
# Now nicely zero padded
items_df.head()

,referenceCode,title,dates,startDate,accessConditions,locations
0,N193-001,Sydney Stock Exchange Stock and Share Lists,1901-01-01 - 1901-03-01,1901-01-01,NaN,NaN
1,N193-002,Sydney Stock Exchange Stock and Share Lists,1901-04-01 - 1901-06-01,1901-04-01,NaN,NaN
2,N193-003,Sydney Stock Exchange Stock and Share Lists,1901-07-01 - 1901-09-01,1901-07-01,NaN,NaN
3,N193-004,Sydney Stock Exchange Stock and Share Lists,1901-10-01 - 1901-12-01,1901-10-01,NaN,NaN
4,N193-005,Sydney Stock Exchange Stock and Share Lists,1902-01-01 - 1902-03-01,1902-01-01,NaN,NaN


In [14]:
# There's a startDate coulmn in the item list, but not a separate endDate.
# Let's extract the endDate from the dates string and create a new column for it.
items_df['endDate'] = items_df['dates'].str.slice(13)

In [15]:
# Extract year from the startDate and create a new column for it.
items_df['year'] = items_df['startDate'].str.slice(0,4)

In [17]:
dates_df = items_df[['referenceCode', 'startDate', 'endDate', 'year']]

In [18]:
# Extract the referenceCode from the directory in the file list and create a new column.
files_df['referenceCode'] = files_df['directory'].str.slice(8,16)

In [19]:
files_df.shape

(72932, 4)

In [20]:
# Ok, now we can use the 'referenceCode' column to link and merge both lists.
new_df = pd.merge(files_df, dates_df, on='referenceCode')

In [21]:
# So now each individual image file has a year!
new_df.head()

,directory,name,path,referenceCode,startDate,endDate,year
0,AU NBAC N193-001/,N193-001_0001.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...,N193-001,1901-01-01,1901-03-01,1901
1,AU NBAC N193-001/,N193-001_0002.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...,N193-001,1901-01-01,1901-03-01,1901
2,AU NBAC N193-001/,N193-001_0003.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...,N193-001,1901-01-01,1901-03-01,1901
3,AU NBAC N193-001/,N193-001_0004.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...,N193-001,1901-01-01,1901-03-01,1901
4,AU NBAC N193-001/,N193-001_0005.tif,Shared/ANU-Library/Sydney Stock Exchange 1901-...,N193-001,1901-01-01,1901-03-01,1901


In [22]:
# Save as a CSV file
new_df.to_csv('files_with_dates.csv', index=False)

In [23]:
new_df['year'].value_counts()

1941    2397
1940    2374
1939    2338
1938    2318
1937    2149
1949    1937
1942    1690
1948    1681
1943    1645
1944    1640
1947    1601
1945    1589
1936    1538
1935    1530
1946    1437
1934    1368
1904    1321
1909    1319
1907    1318
1903    1314
1905    1313
1912    1312
1908    1310
1920    1309
1902    1306
1911    1304
1928    1303
1931    1301
1913    1299
1932    1299
1930    1298
1906    1298
1910    1298
1933    1297
1929    1296
1926    1295
1924    1294
1927    1293
1919    1289
1915    1287
1922    1287
1925    1286
1923    1284
1918    1283
1916    1283
1917    1280
1921    1276
1901    1272
1914    1123
1950    1053
Name: year, dtype: int64